In [10]:
pip install psycopg2-binary requests beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install requests psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [25]:
"""
SCRAPER AFFARE.TN - VERSION FINALE PAGINATION NUMÉRIQUE
Clique sur les numéros de pages (1, 2, 3, 4, 5...)
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import psycopg2
import time
import re

# =========================
# DB CONFIG
# =========================
DB_CONFIG = {
    'host': 'localhost',
    'database': 'scraping_affare',
    'user': 'postgres',
    'password': 'samar'
}

conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()
print("✅ Connexion PostgreSQL réussie\n")

# =========================
# SELENIUM CONFIG
# =========================
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 15)

# =========================
# CONFIG
# =========================
MAX_PAGES = 50      # Nombre de pages à scraper
DELAY = 3           # Délai entre pages

stats = {
    'pages_scrapees': 0,
    'annonces_trouvees': 0,
    'nouvelles': 0,
    'doublons': 0,
    'erreurs': 0
}

# =========================
# FONCTIONS
# =========================

def attendre_chargement():
    """Attendre que la page soit chargée"""
    time.sleep(3)
    # Attendre que les cartes d'annonces soient visibles
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/annonce/']")))
    except:
        pass

def extraire_liens_annonces():
    """Extraire tous les liens d'annonces de la page"""
    
    # Scroll un peu pour charger les images
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    
    liens = set()
    
    # Essayer plusieurs sélecteurs
    selecteurs = [
        "a[href*='/annonce/']",
        ".card a[href*='/annonce/']",
        "[class*='card'] a[href*='/annonce/']",
        "[class*='item'] a[href*='/annonce/']"
    ]
    
    for selecteur in selecteurs:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selecteur)
            for elem in elements:
                href = elem.get_attribute("href")
                if href and '/annonce/' in href and 'affare.tn' in href:
                    liens.add(href)
        except:
            continue
    
    return liens

def cliquer_page_suivante(numero_page):
    """Cliquer sur le numéro de page suivant"""
    
    print(f"  🔍 Recherche page {numero_page}...")
    
    try:
        # Scroll vers le bas pour voir la pagination
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Chercher la pagination
        # Essayer de trouver le lien avec le numéro
        selecteurs = [
            f"a:contains('{numero_page}')",
            f"[class*='pagination'] a:contains('{numero_page}')",
            f".pagination a:contains('{numero_page}')",
        ]
        
        # Méthode 1: Chercher tous les liens de pagination
        try:
            # Trouver tous les éléments de pagination
            pagination_links = driver.find_elements(By.CSS_SELECTOR, 
                "[class*='pagination'] a, .pagination a, [class*='pager'] a, nav a")
            
            for link in pagination_links:
                text = link.text.strip()
                if text == str(numero_page):
                    # Scroll vers l'élément
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link)
                    time.sleep(1)
                    
                    # Cliquer
                    try:
                        link.click()
                    except:
                        # Si clic normal échoue, utiliser JavaScript
                        driver.execute_script("arguments[0].click();", link)
                    
                    print(f"  ✅ Cliqué sur page {numero_page}")
                    attendre_chargement()
                    return True
        except Exception as e:
            print(f"  ⚠️  Méthode 1 échouée: {e}")
        
        # Méthode 2: Construire l'URL directement
        try:
            url_actuelle = driver.current_url
            
            # Essayer différents formats d'URL
            formats_url = [
                f"{url_actuelle}?page={numero_page}",
                f"{url_actuelle}&page={numero_page}",
                url_actuelle.split('?')[0] + f"?page={numero_page}",
            ]
            
            for url in formats_url:
                print(f"  🔗 Essai URL: {url[:60]}...")
                driver.get(url)
                attendre_chargement()
                
                # Vérifier si on a bien changé de page
                liens_actuels = extraire_liens_annonces()
                if len(liens_actuels) > 0:
                    print(f"  ✅ Page {numero_page} chargée via URL")
                    return True
        
        except Exception as e:
            print(f"  ⚠️  Méthode 2 échouée: {e}")
        
        # Méthode 3: Cliquer sur bouton "Suivant" > ou »
        try:
            boutons_suivant = driver.find_elements(By.CSS_SELECTOR, 
                "a.next, button.next, a:contains('>'), a:contains('›'), a:contains('»')")
            
            for btn in boutons_suivant:
                if btn.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                    time.sleep(1)
                    
                    try:
                        btn.click()
                    except:
                        driver.execute_script("arguments[0].click();", btn)
                    
                    print(f"  ✅ Cliqué sur 'Suivant'")
                    attendre_chargement()
                    return True
        except:
            pass
        
        print(f"  ❌ Page {numero_page} non trouvée")
        return False
        
    except Exception as e:
        print(f"  ❌ Erreur: {e}")
        return False

def scraper_annonce(lien):
    """Scraper une annonce"""
    
    try:
        driver.get(lien)
        time.sleep(2)
        
        # Vérifier si existe
        cursor.execute("SELECT id FROM annonces_affare WHERE lien = %s", (lien,))
        if cursor.fetchone():
            print("  ⏭️  Déjà dans la base")
            stats['doublons'] += 1
            return
        
        # Données
        data = {
            'titre': None,
            'prix': None,
            'superficie': None,
            'prix_m2': None,
            'type_bien': 'Immobilier',
            'categorie': None,
            'gouvernorat': None,
            'ville': None,
            'adresse': None,
            'date_publication': None,
            'description': None,
            'lien': lien,
            'source': 'affare.tn'
        }
        
        body_text = driver.find_element(By.TAG_NAME, 'body').text
        
        # TITRE
        try:
            data['titre'] = driver.find_element(By.TAG_NAME, "h1").text.strip()
        except:
            pass
        
        # PRIX
        match = re.search(r'([\d\s]+)\s*DT', body_text, re.IGNORECASE)
        if match:
            data['prix'] = match.group(1).replace(' ', '') + " DT"
        
        # SUPERFICIE
        match = re.search(r'(\d+)\s*m[²2]', body_text, re.IGNORECASE)
        if match:
            data['superficie'] = match.group(1) + " m²"
        
        # GOUVERNORAT/VILLE (depuis le texte "Hammamet, Nabeul")
        gouvernorats = {
            'tunis': 'Tunis', 'nabeul': 'Nabeul', 'sousse': 'Sousse', 
            'sfax': 'Sfax', 'ariana': 'Ariana', 'ben arous': 'Ben Arous',
            'monastir': 'Monastir', 'mahdia': 'Mahdia', 'bizerte': 'Bizerte',
            'gabès': 'Gabès', 'médenine': 'Médenine', 'kairouan': 'Kairouan',
            'raoued': 'Ariana', 'hammamet': 'Nabeul'
        }
        
        text_lower = body_text.lower()
        for key, val in gouvernorats.items():
            if key in text_lower:
                data['gouvernorat'] = val
                break
        
        # Villes
        villes = ['Hammamet', 'La Marsa', 'Sousse', 'Sfax', 'Monastir', 'Nabeul', 
                 'Ariana', 'Tunis', 'Carthage', 'Gammarth']
        for ville in villes:
            if ville in body_text:
                data['ville'] = ville
                break
        
        # CATÉGORIE
        categories = ['Terrains', 'Fermes', 'Appartements', 'Maisons', 'Villas', 
                     'Bureaux', 'Locaux', 'Duplex']
        for cat in categories:
            if cat in body_text:
                data['categorie'] = cat
                break
        
        # DATE
        if "aujourd'hui" in text_lower:
            data['date_publication'] = "Publiée aujourd'hui"
        elif 'hier' in text_lower:
            data['date_publication'] = "Publiée hier"
        
        # DESCRIPTION
        try:
            desc_elem = driver.find_element(By.CSS_SELECTOR, '[class*="description"], .description, [class*="detail"]')
            data['description'] = desc_elem.text[:1000]
        except:
            data['description'] = body_text[:1000]
        
        # Afficher
        print(f"  📋 {data['titre'][:40] if data['titre'] else 'N/A'}...")
        print(f"  💰 {data['prix']} | 📏 {data['superficie']} | 📍 {data['gouvernorat']}")
        
        # Sauvegarder
        cursor.execute("""
            INSERT INTO annonces_affare (
                titre, prix, superficie, prix_m2, type_bien,
                categorie, gouvernorat, ville, adresse, 
                date_publication, description, lien, source
            )
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT (lien) DO NOTHING
        """, (
            data['titre'], data['prix'], data['superficie'], data['prix_m2'],
            data['type_bien'], data['categorie'], data['gouvernorat'], data['ville'],
            data['adresse'], data['date_publication'], data['description'],
            data['lien'], data['source']
        ))
        
        conn.commit()
        print(f"  ✅ Sauvegardée")
        stats['nouvelles'] += 1
        
    except Exception as e:
        print(f"  ❌ Erreur: {e}")
        stats['erreurs'] += 1
        conn.rollback()

# =========================
# MAIN - SCRAPING COMPLET
# =========================

print("="*70)
print("🚀 SCRAPER AFFARE.TN - PAGINATION NUMÉRIQUE")
print("="*70)

URL_BASE = "https://www.affare.tn/petites-annonces/tunisie/immobilier"

# Aller sur la première page
print(f"\n📍 Accès à la page 1...")
driver.get(URL_BASE)
attendre_chargement()

# Collecter toutes les annonces de toutes les pages
toutes_annonces = set()

for page_num in range(1, MAX_PAGES + 1):
    
    print(f"\n{'='*70}")
    print(f"📑 PAGE {page_num}/{MAX_PAGES}")
    print(f"{'='*70}")
    
    # Si ce n'est pas la page 1, cliquer sur le numéro
    if page_num > 1:
        if not cliquer_page_suivante(page_num):
            print(f"⚠️  Impossible d'accéder à la page {page_num}, arrêt")
            break
    
    stats['pages_scrapees'] += 1
    
    # Extraire les annonces de cette page
    print(f"  📋 Extraction des annonces...")
    liens_page = extraire_liens_annonces()
    
    nouveaux = liens_page - toutes_annonces
    toutes_annonces.update(nouveaux)
    
    print(f"  ✅ {len(nouveaux)} nouvelles annonces trouvées")
    print(f"  📊 Total cumulé: {len(toutes_annonces)}")
    
    stats['annonces_trouvees'] += len(nouveaux)
    
    time.sleep(DELAY)

print(f"\n{'='*70}")
print(f"✅ DÉCOUVERTE TERMINÉE: {len(toutes_annonces)} annonces")
print(f"{'='*70}\n")

# Phase 2: Scraping des annonces
print(f"{'='*70}")
print("🏗️  PHASE 2: SCRAPING DES ANNONCES")
print(f"{'='*70}\n")

annonces_liste = list(toutes_annonces)

for idx, lien in enumerate(annonces_liste, 1):
    print(f"\n[{idx}/{len(annonces_liste)}] 📄 {lien[:60]}...")
    scraper_annonce(lien)
    time.sleep(1)
    
    if idx % 10 == 0:
        print(f"\n📊 [{idx}/{len(annonces_liste)}] Nouvelles: {stats['nouvelles']} | Doublons: {stats['doublons']}\n")

# Résultats
print(f"\n{'='*70}")
print("📊 RÉSULTATS FINAUX")
print(f"{'='*70}")
print(f"📑 Pages explorées: {stats['pages_scrapees']}")
print(f"🔗 Annonces découvertes: {stats['annonces_trouvees']}")
print(f"🆕 Nouvelles annonces: {stats['nouvelles']}")
print(f"♻️  Doublons: {stats['doublons']}")
print(f"❌ Erreurs: {stats['erreurs']}")
print(f"{'='*70}")

cursor.execute("SELECT COUNT(*) FROM annonces_affare")
total = cursor.fetchone()[0]
print(f"\n💾 Total dans la base: {total} annonces")

cursor.execute("""
    SELECT gouvernorat, COUNT(*) as nb 
    FROM annonces_affare 
    WHERE gouvernorat IS NOT NULL 
    GROUP BY gouvernorat 
    ORDER BY nb DESC 
    LIMIT 5
""")
print(f"\n🏆 Top 5 gouvernorats:")
for row in cursor.fetchall():
    print(f"   {row[0]}: {row[1]}")

cursor.execute("""
    SELECT categorie, COUNT(*) as nb 
    FROM annonces_affare 
    WHERE categorie IS NOT NULL 
    GROUP BY categorie 
    ORDER BY nb DESC
""")
print(f"\n📊 Par catégorie:")
for row in cursor.fetchall():
    print(f"   {row[0]}: {row[1]}")

print("\n✅ SCRAPING TERMINÉ!")

driver.quit()
cursor.close()
conn.close()

✅ Connexion PostgreSQL réussie

🚀 SCRAPER AFFARE.TN - PAGINATION NUMÉRIQUE

📍 Accès à la page 1...

📑 PAGE 1/50
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 30

📑 PAGE 2/50
  🔍 Recherche page 2...
  ✅ Cliqué sur page 2
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 60

📑 PAGE 3/50
  🔍 Recherche page 3...
  ✅ Cliqué sur page 3
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 90

📑 PAGE 4/50
  🔍 Recherche page 4...
  ✅ Cliqué sur page 4
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 120

📑 PAGE 5/50
  🔍 Recherche page 5...
  ✅ Cliqué sur page 5
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 150

📑 PAGE 6/50
  🔍 Recherche page 6...
  ✅ Cliqué sur page 6
  📋 Extraction des annonces...
  ✅ 30 nouvelles annonces trouvées
  📊 Total cumulé: 180

📑 PAGE 7/50
  🔍 Recherche page 7...
  ✅ Cliqué sur page 7
  📋 Extra